In [1]:
import chess.pgn
import pandas as pd
from sqlalchemy import create_engine, text
import os
from datetime import datetime

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [2]:
#Database configuration
DB_USER = os.getenv('USER')
DB_NAME = 'chess_app'
DATABASE_URL = f'postgresql://{DB_USER}:@localhost:5432/{DB_NAME}'

#Create Engine
engine = create_engine(DATABASE_URL, echo=False)

#Test Connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM games;"))
    count = result.fetchone()[0]
    print(f"✅ Connected to database!")
    print(f"Current games in database: {count}")

✅ Connected to database!
Current games in database: 1200


In [3]:
def parse_pgn_file(pgn_file_path, max_games=100):
    """
    Parse PGN file and extract game data
    
    Args:
        pgn_file_path: Path to PGN file
        max_games: Maximum number of games to parse
    
    Returns:
        DataFrame with game data
    """
    games_data = []
    
    with open(pgn_file_path) as pgn:
        game_count = 0
        
        while game_count < max_games:
            game = chess.pgn.read_game(pgn)
            
            if game is None:
                break
            
            headers = game.headers
            
            # Extract game data
            game_data = {
                'white_player': headers.get('White', 'Unknown'),
                'black_player': headers.get('Black', 'Unknown'),
                'white_elo': int(headers.get('WhiteElo', 0)) if headers.get('WhiteElo', '').isdigit() else None,
                'black_elo': int(headers.get('BlackElo', 0)) if headers.get('BlackElo', '').isdigit() else None,
                'result': headers.get('Result', '*'),
                'game_date': headers.get('Date', '').replace('.', '-') if headers.get('Date') else None,
                'opening_name': headers.get('Opening', 'Unknown'),
                'pgn_moves': game.board().variation_san(game.mainline_moves()),
                'source': 'lichess'
            }
            
            games_data.append(game_data)
            game_count += 1
            
            # Progress indicator
            if game_count % 10 == 0:
                print(f"Parsed {game_count} games...", end='\r')
    
    print(f"\n✅ Parsed {len(games_data)} games total!")
    return pd.DataFrame(games_data)

print("✅ Parse function defined!")

✅ Parse function defined!


In [4]:
import os

# Get the absolute path to your project root
project_root = os.path.expanduser('~/chess_project')

# Path to your sample PGN file (absolute path)
pgn_path = os.path.join(project_root, 'data/raw/sample_1000_games.pgn')

# Verify the file exists
if os.path.exists(pgn_path):
    print(f"✅ Found file: {pgn_path}")
    file_size = os.path.getsize(pgn_path) / (1024 * 1024)  # Convert to MB
    print(f"File size: {file_size:.2f} MB")
else:
    print(f"❌ File not found: {pgn_path}")
    print("\nLet's check what's in data/raw/:")
    raw_dir = os.path.join(project_root, 'data/raw')
    if os.path.exists(raw_dir):
        print(os.listdir(raw_dir))
    else:
        print("data/raw directory doesn't exist!")

# Parse 100 games first (to test)
print("\nStarting to parse PGN file...")
games_df = parse_pgn_file(pgn_path, max_games=100)

# Display first few games
print("\nFirst 5 games:")
games_df.head()

✅ Found file: /Users/miguel/chess_project/data/raw/sample_1000_games.pgn
File size: 2.23 MB

Starting to parse PGN file...
Parsed 100 games...
✅ Parsed 100 games total!

First 5 games:


,white_player,black_player,white_elo,black_elo,result,game_date,opening_name,pgn_moves,source
0,Detectie,youssefaymn,1176,1155,1-0,2024-01-01,Italian Game: Anti-Fried Liver Defense,1. e4 e5 2. Nf3 Nc6 3. Bc4 h6 4. O-O Nf6 5. d3...,lichess
1,LionsAK32,Vusal_1990,732,763,0-1,2024-01-01,Vienna Game: Stanley Variation,1. e4 e5 2. Bc4 Nf6 3. Nc3 d6 4. Qf3 Be7 5. d4...,lichess
2,DRAGONKING54,fanydacute,1783,1784,1-0,2024-01-01,Dutch Defense: Fianchetto Attack,1. d4 f5 2. g3 Nf6 3. Bg2 e6 4. Nf3 c5 5. c3 N...,lichess
3,TRUFFE68,Davigoal,1491,1495,0-1,2024-01-01,Polish Opening,1. b4 e6 2. Bb2 b6 3. Nf3 Bb7 4. g3 f6 5. c3 N...,lichess
4,rodrigorramos,ThomasTuchel,1391,1381,0-1,2024-01-01,Modern Defense: Standard Line,1. d4 g6 2. e4 Bg7 3. Nc3 Na6,lichess


In [5]:
# Basic statistics
print("=" * 60)
print("DATASET STATISTICS")
print("=" * 60)

print(f"\n📊 Total games parsed: {len(games_df)}")

print(f"\n🏆 Results distribution:")
print(games_df['result'].value_counts())

print(f"\n⭐ ELO Ratings:")
print(f"   White average: {games_df['white_elo'].mean():.0f}")
print(f"   Black average: {games_df['black_elo'].mean():.0f}")
print(f"   Highest rated: {games_df[['white_elo', 'black_elo']].max().max()}")
print(f"   Lowest rated: {games_df[['white_elo', 'black_elo']].min().min()}")

print(f"\n♟️  Top 10 Most Common Openings:")
print(games_df['opening_name'].value_counts().head(10))

print(f"\n📅 Date range:")
print(f"   From: {games_df['game_date'].min()}")
print(f"   To: {games_df['game_date'].max()}")

DATASET STATISTICS

📊 Total games parsed: 100

🏆 Results distribution:
result
1-0        51
0-1        47
1/2-1/2     2
Name: count, dtype: int64

⭐ ELO Ratings:
   White average: 1655
   Black average: 1659
   Highest rated: 2658
   Lowest rated: 635

♟️  Top 10 Most Common Openings:
opening_name
Van't Kruijs Opening                            5
Horwitz Defense                                 4
Queen's Pawn Game: Accelerated London System    3
Modern Defense                                  3
Queen's Pawn Game                               3
Italian Game: Anti-Fried Liver Defense          2
Sicilian Defense: Franco-Sicilian Variation     2
Indian Defense                                  2
Owen Defense                                    2
Sicilian Defense: Modern Variations             2
Name: count, dtype: int64

📅 Date range:
   From: 2024-01-01
   To: 2024-01-01


In [6]:
# Load games into database
print("=" * 60)
print("LOADING DATA INTO POSTGRESQL")
print("=" * 60)

try:
    # Insert games into database
    games_df.to_sql('games', engine, if_exists='append', index=False)
    
    print(f"\n✅ Successfully loaded {len(games_df)} games into database!")
    
    # Verify
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM games;"))
        total = result.fetchone()[0]
        print(f"📊 Total games now in database: {total}")
        
except Exception as e:
    print(f"❌ Error: {e}")

LOADING DATA INTO POSTGRESQL

✅ Successfully loaded 100 games into database!
📊 Total games now in database: 1300


In [7]:
# Query games from the database
print("=" * 60)
print("QUERYING DATABASE")
print("=" * 60)

query = """
SELECT 
    game_id,
    white_player,
    black_player,
    white_elo,
    black_elo,
    result,
    opening_name
FROM games
ORDER BY (white_elo + black_elo) DESC
LIMIT 10;
"""

print("\n🏆 Top 10 Highest Rated Games:\n")
result_df = pd.read_sql(query, engine)
result_df

QUERYING DATABASE

🏆 Top 10 Highest Rated Games:



,game_id,white_player,black_player,white_elo,black_elo,result,opening_name
0,183,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
1,1283,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
2,83,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
3,283,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
4,1009,albertosauruso,ori_1996,2575,2612,0-1,Caro-Kann Defense
5,912,healer69,Misteriochess123,2539,2606,1-0,Zukertort Opening: Tennison Gambit
6,1063,Sepehrov,teikaluk,2535,2573,1-0,Nimzo-Larsen Attack: Classical Variation
7,614,HejDu,Cat_in_hat,2102,3003,0-1,Sicilian Defense: Smith-Morra Gambit
8,808,Molot_ua,refresher,2515,2538,1-0,Mieses Opening: Reversed Rat
9,916,volkov_denis,lancemanalo77,2544,2496,1-0,French Defense: Steiner Variation


In [8]:
# Query games from the database
print("=" * 60)
print("QUERYING DATABASE")
print("=" * 60)

query = """
SELECT 
    game_id,
    white_player,
    black_player,
    white_elo,
    black_elo,
    result,
    opening_name
FROM games
ORDER BY (white_elo + black_elo) DESC
LIMIT 10;
"""

print("\n🏆 Top 10 Highest Rated Games:\n")
result_df = pd.read_sql(query, engine)
result_df

QUERYING DATABASE

🏆 Top 10 Highest Rated Games:



,game_id,white_player,black_player,white_elo,black_elo,result,opening_name
0,183,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
1,1283,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
2,83,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
3,283,LighTomorrowithTODAY,ShYarEx,2556,2658,1-0,"Queen's Pawn Game: Veresov Attack, Classical D..."
4,1009,albertosauruso,ori_1996,2575,2612,0-1,Caro-Kann Defense
5,912,healer69,Misteriochess123,2539,2606,1-0,Zukertort Opening: Tennison Gambit
6,1063,Sepehrov,teikaluk,2535,2573,1-0,Nimzo-Larsen Attack: Classical Variation
7,614,HejDu,Cat_in_hat,2102,3003,0-1,Sicilian Defense: Smith-Morra Gambit
8,808,Molot_ua,refresher,2515,2538,1-0,Mieses Opening: Reversed Rat
9,916,volkov_denis,lancemanalo77,2544,2496,1-0,French Defense: Steiner Variation


In [9]:
def extract_positions_from_game(game_id, pgn_moves_str):
    """
    Extract individual positions from a game
    
    Args:
        game_id: The game ID from database
        pgn_moves_str: String of moves like "1. e4 e5 2. Nf3 Nc6..."
    
    Returns:
        List of position dictionaries
    """
    import io
    
    positions = []
    
    try:
        # Create a game from the moves
        game = chess.pgn.read_game(io.StringIO(f"[Event \"\"]\n\n{pgn_moves_str}"))
        
        if game is None:
            return positions
        
        board = game.board()
        move_number = 0
        
        # Iterate through all moves
        for move in game.mainline_moves():
            move_number += 1
            
            # Record position BEFORE the move
            fen = board.fen()
            move_san = board.san(move)
            
            position_data = {
                'game_id': game_id,
                'move_number': move_number,
                'fen': fen,
                'move_played': move_san
            }
            
            positions.append(position_data)
            
            # Make the move
            board.push(move)
    
    except Exception as e:
        print(f"Error processing game {game_id}: {e}")
    
    return positions

print("✅ Position extraction function defined!")

✅ Position extraction function defined!


In [10]:
print("=" * 60)
print("EXTRACTING POSITIONS FROM GAMES")
print("=" * 60)

# Get first 10 games from database
query = """
SELECT game_id, pgn_moves
FROM games
LIMIT 10;
"""

games_to_process = pd.read_sql(query, engine)

all_positions = []

for idx, row in games_to_process.iterrows():
    game_id = row['game_id']
    pgn_moves = row['pgn_moves']
    
    positions = extract_positions_from_game(game_id, pgn_moves)
    all_positions.extend(positions)
    
    print(f"Game {game_id}: extracted {len(positions)} positions", end='\r')

print(f"\n\n✅ Total positions extracted: {len(all_positions)}")

# Convert to DataFrame
positions_df = pd.DataFrame(all_positions)

print("\nFirst 5 positions:")
positions_df.head()

EXTRACTING POSITIONS FROM GAMES
Game 10: extracted 114 positions

✅ Total positions extracted: 673

First 5 positions:


,game_id,move_number,fen,move_played
0,1,1,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,e4
1,1,2,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,e5
2,1,3,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,Nf3
3,1,4,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,Nc6
4,1,5,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,Bc4


In [11]:
print("=" * 60)
print("LOADING POSITIONS INTO DATABASE")
print("=" * 60)

try:
    # Insert positions into database
    positions_df.to_sql('positions', engine, if_exists='append', index=False)
    
    print(f"\n✅ Successfully loaded {len(positions_df)} positions into database!")
    
    # Verify
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM positions;"))
        total = result.fetchone()[0]
        print(f"📊 Total positions now in database: {total}")
        
except Exception as e:
    print(f"❌ Error: {e}")

LOADING POSITIONS INTO DATABASE
❌ Error: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "positions_game_id_move_number_key"
DETAIL:  Key (game_id, move_number)=(1, 1) already exists.

[SQL: INSERT INTO positions (game_id, move_number, fen, move_played) VALUES (%(game_id__0)s, %(move_number__0)s, %(fen__0)s, %(move_played__0)s), (%(game_id__1)s, %(move_number__1)s, %(fen__1)s, %(move_played__1)s), (%(game_id__2)s, %(move_number__2)s, %(f ... 51772 characters truncated ...  %(move_played__671)s), (%(game_id__672)s, %(move_number__672)s, %(fen__672)s, %(move_played__672)s)]
[parameters: {'move_number__0': 1, 'game_id__0': 1, 'move_played__0': 'e4', 'fen__0': 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1', 'move_number__1': 2, 'game_id__1': 1, 'move_played__1': 'e5', 'fen__1': 'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1', 'move_number__2': 3, 'game_id__2': 1, 'move_played__2': 'Nf3', 'fen__2': 'rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP

In [12]:
# Query some positions
query = """
SELECT 
    p.position_id,
    p.game_id,
    p.move_number,
    p.move_played,
    p.fen,
    g.white_player,
    g.black_player
FROM positions p
JOIN games g ON p.game_id = g.game_id
LIMIT 10;
"""

print("Sample positions from database:\n")
pd.read_sql(query, engine)

Sample positions from database:



,position_id,game_id,move_number,move_played,fen,white_player,black_player
0,1,1,1,e4,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,Detectie,youssefaymn
1,2,1,2,e5,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,Detectie,youssefaymn
2,3,1,3,Nf3,rnbqkbnr/pppp1ppp/8/4p3/4P3/8/PPPP1PPP/RNBQKBN...,Detectie,youssefaymn
3,4,1,4,Nc6,rnbqkbnr/pppp1ppp/8/4p3/4P3/5N2/PPPP1PPP/RNBQK...,Detectie,youssefaymn
4,5,1,5,Bc4,r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNB...,Detectie,youssefaymn
5,6,1,6,h6,r1bqkbnr/pppp1ppp/2n5/4p3/2B1P3/5N2/PPPP1PPP/R...,Detectie,youssefaymn
6,7,1,7,O-O,r1bqkbnr/pppp1pp1/2n4p/4p3/2B1P3/5N2/PPPP1PPP/...,Detectie,youssefaymn
7,8,1,8,Nf6,r1bqkbnr/pppp1pp1/2n4p/4p3/2B1P3/5N2/PPPP1PPP/...,Detectie,youssefaymn
8,9,1,9,d3,r1bqkb1r/pppp1pp1/2n2n1p/4p3/2B1P3/5N2/PPPP1PP...,Detectie,youssefaymn
9,10,1,10,d6,r1bqkb1r/pppp1pp1/2n2n1p/4p3/2B1P3/3P1N2/PPP2P...,Detectie,youssefaymn


In [13]:
# Load the remaining 900 games (we already have 100)
print("=" * 60)
print("SCALING UP: LOADING REMAINING 900 GAMES")
print("=" * 60)

# Parse 900 more games (starting from where we left off)
print("\nParsing games 101-1000...")
games_df_new = parse_pgn_file(pgn_path, max_games=1000)

# Remove the first 100 (we already have those)
games_df_new = games_df_new.iloc[100:]

print(f"\n✅ Parsed {len(games_df_new)} new games!")
print(f"New games ELO range: {games_df_new['white_elo'].min()}-{games_df_new['white_elo'].max()}")

# Load into database
try:
    games_df_new.to_sql('games', engine, if_exists='append', index=False)
    print(f"✅ Successfully loaded {len(games_df_new)} new games into database!")
    
    # Verify total
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM games;"))
        total = result.fetchone()[0]
        print(f"📊 Total games now in database: {total}")
except Exception as e:
    print(f"❌ Error: {e}")

SCALING UP: LOADING REMAINING 900 GAMES

Parsing games 101-1000...
Parsed 1000 games...
✅ Parsed 1000 games total!

✅ Parsed 900 new games!
New games ELO range: 571-2575
✅ Successfully loaded 900 new games into database!
📊 Total games now in database: 2200


In [40]:
print("=" * 60)
print("BATCH PROCESSING - EXTRACT POSITIONS FOR NEW GAMES")
print("=" * 60)

# Find games that don't have positions yet
query = """
SELECT g.game_id, g.pgn_moves
FROM games g
LEFT JOIN positions p ON g.game_id = p.game_id
WHERE p.position_id IS NULL
ORDER BY g.game_id
LIMIT 100;
"""

games_batch = pd.read_sql(query, engine)
print(f"\nGames without positions: {len(games_batch)}")

if len(games_batch) == 0:
    print("✅ All games already have positions!")
else:
    positions_batch = []
    
    for idx, row in games_batch.iterrows():
        positions = extract_positions_from_game(row['game_id'], row['pgn_moves'])
        positions_batch.extend(positions)
        
        if (idx + 1) % 10 == 0:
            print(f"  Processed {idx + 1}/{len(games_batch)} games | {len(positions_batch)} positions")
    
    print(f"\n✅ Extracted {len(positions_batch)} positions from {len(games_batch)} games")
    
    if len(positions_batch) > 0:
        print("Loading into database...")
        pd.DataFrame(positions_batch).to_sql('positions', engine, if_exists='append', index=False)
        print("✅ Batch loaded!")
        
        with engine.connect() as conn:
            total = conn.execute(text("SELECT COUNT(*) FROM positions;")).fetchone()[0]
            print(f"📊 Total positions in database: {total}")

BATCH PROCESSING - EXTRACT POSITIONS FOR NEW GAMES

Games without positions: 10
  Processed 10/10 games | 0 positions

✅ Extracted 0 positions from 10 games


In [39]:
with engine.connect() as conn:
    games_total = conn.execute(text("SELECT COUNT(*) FROM games;")).fetchone()[0]
    positions_total = conn.execute(text("SELECT COUNT(*) FROM positions;")).fetchone()[0]
    games_with_pos = conn.execute(text("SELECT COUNT(DISTINCT game_id) FROM positions;")).fetchone()[0]
    
    print(f"📊 Progress:")
    print(f"   Total games: {games_total}")
    print(f"   Games with positions: {games_with_pos}")
    print(f"   Games remaining: {games_total - games_with_pos}")
    print(f"   Total positions: {positions_total}")

📊 Progress:
   Total games: 2200
   Games with positions: 2190
   Games remaining: 10
   Total positions: 148320


In [41]:
# Check what those 10 games are
query = """
SELECT g.game_id, g.white_player, g.black_player, g.result, g.pgn_moves
FROM games g
LEFT JOIN positions p ON g.game_id = p.game_id
WHERE p.position_id IS NULL
LIMIT 10;
"""

empty_games = pd.read_sql(query, engine)
print("Games with no positions:")
print(empty_games)

Games with no positions:
   game_id       white_player      black_player result pgn_moves
0     1027        DrRaulstein             eldry    0-1          
1     1200         jeanrobi84       marcosdeath    0-1          
2     2031  JoseSantosMedrano         Ian030309    0-1          
3      412      fennecfoxlevi  G4nd4lfl4l3g3nd3    0-1          
4     1412      fennecfoxlevi  G4nd4lfl4l3g3nd3    0-1          
5     2027        DrRaulstein             eldry    0-1          
6     1031  JoseSantosMedrano         Ian030309    0-1          
7      329      TeodoroMTalon           m3gumin    0-1          
8     2200         jeanrobi84       marcosdeath    0-1          
9     1329      TeodoroMTalon           m3gumin    0-1          


In [42]:
print("=" * 60)
print("🎉 DATA PIPELINE COMPLETE - FINAL STATISTICS")
print("=" * 60)

with engine.connect() as conn:
    # Games stats
    games_total = conn.execute(text("SELECT COUNT(*) FROM games;")).fetchone()[0]
    
    # Positions stats
    positions_total = conn.execute(text("SELECT COUNT(*) FROM positions;")).fetchone()[0]
    games_with_positions = conn.execute(text("SELECT COUNT(DISTINCT game_id) FROM positions;")).fetchone()[0]
    avg_positions_per_game = positions_total / games_with_positions if games_with_positions > 0 else 0
    
    # ELO stats
    elo_query = """
    SELECT 
        MIN(LEAST(white_elo, black_elo)) as min_elo,
        MAX(GREATEST(white_elo, black_elo)) as max_elo,
        AVG((white_elo + black_elo)/2) as avg_elo
    FROM games
    WHERE white_elo IS NOT NULL AND black_elo IS NOT NULL;
    """
    elo_stats = pd.read_sql(elo_query, engine).iloc[0]
    
    print(f"\n📊 GAMES:")
    print(f"   Total games in database: {games_total:,}")
    print(f"   Games with positions: {games_with_positions:,}")
    print(f"   Games without positions: {games_total - games_with_positions:,} (abandoned)")
    
    print(f"\n📊 POSITIONS (ML Training Data):")
    print(f"   Total positions: {positions_total:,}")
    print(f"   Avg positions per game: {avg_positions_per_game:.1f} moves")
    
    print(f"\n⭐ ELO RANGE:")
    print(f"   Lowest: {int(elo_stats['min_elo']):,}")
    print(f"   Highest: {int(elo_stats['max_elo']):,}")
    print(f"   Average: {int(elo_stats['avg_elo']):,}")
    
    # Top openings
    print(f"\n♟️  TOP 10 OPENINGS:")
    top_openings = pd.read_sql("""
        SELECT opening_name, COUNT(*) as count
        FROM games
        GROUP BY opening_name
        ORDER BY count DESC
        LIMIT 10;
    """, engine)
    for idx, row in top_openings.iterrows():
        print(f"   {idx+1}. {row['opening_name']}: {row['count']} games")

print("\n" + "=" * 60)
print("✅ Phase C: COMPLETE!")
print("✅ You now have ~148,000 chess positions for ML training!")
print("✅ Ready for Phase D: Exploratory Data Analysis!")
print("✅ Ready for Phase E: ML Model Training!")
print("=" * 60)

🎉 DATA PIPELINE COMPLETE - FINAL STATISTICS

📊 GAMES:
   Total games in database: 2,200
   Games with positions: 2,190
   Games without positions: 10 (abandoned)

📊 POSITIONS (ML Training Data):
   Total positions: 148,320
   Avg positions per game: 67.7 moves

⭐ ELO RANGE:
   Lowest: 568
   Highest: 3,003
   Average: 1,649

♟️  TOP 10 OPENINGS:
   1. Queen's Pawn Game: 56 games
   2. Van't Kruijs Opening: 48 games
   3. Modern Defense: 46 games
   4. Queen's Pawn Game: Accelerated London System: 38 games
   5. Owen Defense: 36 games
   6. Horwitz Defense: 36 games
   7. Caro-Kann Defense: 34 games
   8. Philidor Defense: 30 games
   9. Pirc Defense: 28 games
   10. Scandinavian Defense: Mieses-Kotroc Variation: 28 games

✅ Phase C: COMPLETE!
✅ You now have ~148,000 chess positions for ML training!
✅ Ready for Phase D: Exploratory Data Analysis!
✅ Ready for Phase E: ML Model Training!
